In [92]:
%matplotlib inline
import pandas as pd
import os
import numpy as np
import sklearn
import spacy
import en_core_web_sm
nlp = en_core_web_sm.load()

from gensim.models import Phrases
from gensim.models.word2vec import LineSentence

In [93]:
NEWLINE = '\n'
SKIP_FILES = {'cmds'}


def read_files(path):
    for root, dir_names, file_names in os.walk(path):
        for path in dir_names:
            read_files(os.path.join(root, path))
        for file_name in file_names:
            if file_name not in SKIP_FILES:
                file_path = os.path.join(root, file_name)
                if os.path.isfile(file_path):
                    past_header, lines = False, []
                    f = open(file_path, encoding="ANSI")
                    for line in f:
                        if past_header:
                            lines.append(line)
                        elif line == NEWLINE:
                            past_header = True
                    f.close()
                    content = NEWLINE.join(lines)
                    yield content
                    


In [94]:
path =r'C:\Users\osutr_000\Documents\Data\Ops'
list_ = []

for value in read_files(path):
    list_.append(value)

ops_df = pd.DataFrame(data = list_)
ops_df['class']="ops"
len(ops_df)

580

In [95]:
path =r'C:\Users\osutr_000\Documents\Data\Legal'
list_ = []

for value in read_files(path):
    list_.append(value)

legal_df = pd.DataFrame(data = list_)
legal_df['class']="legal"
len(legal_df)

209

In [96]:
path =r'C:\Users\osutr_000\Documents\Data\Accounting'
list_ = []

for value in read_files(path):
    list_.append(value)

act_df = pd.DataFrame(data = list_)
act_df['class']="accounting"
len(act_df)

324

In [97]:
merged_df = ops_df.append(legal_df).append(act_df)
len(merged_df)
merged_df.columns = ['text', 'cat']
merged_df.sort_index().reset_index().head()

,index,text,cat
0,0,"401 W. 33rd \n\nEdmond, OK 73013 \n\nWWW.MIDCO...",ops
1,0,26938 \n\n\n\n002 \n\n\n\n4/27/16 290.60 290. ...,accounting
2,0,"MIDCON Data Services, LLC. \n\n\n\nThis Data D...",legal
3,1,5/25/16 179.17 179.17 \n\n\n\n\n\n5/27/16 0160...,accounting
4,1,,legal


In [98]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from __future__ import print_function
from pprint import pprint
from time import time
import logging

In [99]:
# Flatten data into a (sample, feature) matrix
X = merged_df.text
y = merged_df.cat

# and then split the dataset into two parts
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.5, random_state=0)

In [100]:
# Define a pipeline combining a text feature extractor with a simple
# classifier
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier()),
])

# uncommenting more parameters will give better exploring power but will
# increase processing time in a combinatorial way
parameters = {
    'vect__max_df': (0.5, 0.75, 1.0),
    #'vect__max_features': (None, 5000, 10000, 50000),
    'vect__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
    'tfidf__use_idf': (True, False),
    #'tfidf__norm': ('l1', 'l2'),
    'clf__max_iter': (5,),
    'clf__alpha': (0.00001, 0.000001),
    'clf__penalty': ('l2', 'elasticnet'),
    #'clf__n_iter': (10, 50, 80),
}


In [101]:
print("# Tuning hyper-parameters...")
print()

clf = GridSearchCV(pipeline, parameters)
clf.fit(X_train, y_train)

print("Best parameters set found on development set:")
print()
print(clf.best_params_)
print()
print("Grid scores on development set:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
          % (mean, std * 2, params))

# Tuning hyper-parameters...



c:\Users\osutr_000\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
c:\Users\osutr_000\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
c:\Users\osutr_000\Anaco

c:\Users\osutr_000\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
c:\Users\osutr_000\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
c:\Users\osutr_000\Anaco

c:\Users\osutr_000\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
c:\Users\osutr_000\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
c:\Users\osutr_000\Anaco

c:\Users\osutr_000\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
c:\Users\osutr_000\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
c:\Users\osutr_000\Anaco

...

c:\Users\osutr_000\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
c:\Users\osutr_000\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
c:\Users\osutr_000\Anaco

c:\Users\osutr_000\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
c:\Users\osutr_000\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
c:\Users\osutr_000\Anaco

c:\Users\osutr_000\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
c:\Users\osutr_000\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
c:\Users\osutr_000\Anaco

Best parameters set found on development set:

{'clf__alpha': 1e-05, 'clf__penalty': 'l2', 'tfidf__norm': 'l2', 'tfidf__use_idf': True, 'vect__max_df': 0.5, 'vect__ngram_range': (1, 2)}

Grid scores on development set:

0.989 (+/-0.015) for {'clf__alpha': 1e-05, 'clf__penalty': 'l2', 'tfidf__norm': 'l1', 'tfidf__use_idf': True, 'vect__max_df': 0.5, 'vect__ngram_range': (1, 1)}
0.919 (+/-0.039) for {'clf__alpha': 1e-05, 'clf__penalty': 'l2', 'tfidf__norm': 'l1', 'tfidf__use_idf': True, 'vect__max_df': 0.5, 'vect__ngram_range': (1, 2)}
0.993 (+/-0.005) for {'clf__alpha': 1e-05, 'clf__penalty': 'l2', 'tfidf__norm': 'l1', 'tfidf__use_idf': True, 'vect__max_df': 0.75, 'vect__ngram_range': (1, 1)}
0.921 (+/-0.035) for {'clf__alpha': 1e-05, 'clf__penalty': 'l2', 'tfidf__norm': 'l1', 'tfidf__use_idf': True, 'vect__max_df': 0.75, 'vect__ngram_range': (1, 2)}
0.989 (+/-0.009) for {'clf__alpha': 1e-05, 'clf__penalty': 'l2', 'tfidf__norm': 'l1', 'tfidf__use_idf': True, 'vect__max_df': 1.0, 'vect__

0.989 (+/-0.015) for {'clf__alpha': 1e-06, 'clf__penalty': 'l2', 'tfidf__norm': 'l1', 'tfidf__use_idf': True, 'vect__max_df': 0.5, 'vect__ngram_range': (1, 1)}
0.939 (+/-0.065) for {'clf__alpha': 1e-06, 'clf__penalty': 'l2', 'tfidf__norm': 'l1', 'tfidf__use_idf': True, 'vect__max_df': 0.5, 'vect__ngram_range': (1, 2)}
0.991 (+/-0.010) for {'clf__alpha': 1e-06, 'clf__penalty': 'l2', 'tfidf__norm': 'l1', 'tfidf__use_idf': True, 'vect__max_df': 0.75, 'vect__ngram_range': (1, 1)}
0.923 (+/-0.035) for {'clf__alpha': 1e-06, 'clf__penalty': 'l2', 'tfidf__norm': 'l1', 'tfidf__use_idf': True, 'vect__max_df': 0.75, 'vect__ngram_range': (1, 2)}
0.995 (+/-0.009) for {'clf__alpha': 1e-06, 'clf__penalty': 'l2', 'tfidf__norm': 'l1', 'tfidf__use_idf': True, 'vect__max_df': 1.0, 'vect__ngram_range': (1, 1)}
0.921 (+/-0.041) for {'clf__alpha': 1e-06, 'clf__penalty': 'l2', 'tfidf__norm': 'l1', 'tfidf__use_idf': True, 'vect__max_df': 1.0, 'vect__ngram_range': (1, 2)}
0.987 (+/-0.021) for {'clf__alpha': 1e

In [102]:
print("Detailed classification report on test set:")
print()
y_true, y_pred = y_test, clf.predict(X_test)
print(classification_report(y_true, y_pred))
print()

print("Detailed classification report on training set")
print()
y_true, y_pred = y_train, clf.predict(X_train)
print(classification_report(y_true, y_pred))
print()

Detailed classification report on test set:

             precision    recall  f1-score   support

 accounting       1.00      0.99      1.00       160
      legal       0.98      1.00      0.99       106
        ops       1.00      1.00      1.00       291

avg / total       1.00      1.00      1.00       557


Detailed classification report on training set

             precision    recall  f1-score   support

 accounting       1.00      1.00      1.00       164
      legal       1.00      1.00      1.00       103
        ops       1.00      1.00      1.00       289

avg / total       1.00      1.00      1.00       556




In [127]:
# Flatten data into a (sample, feature) matrix
X = merged_df.text
y = merged_df.cat

# and then split the dataset into two parts
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.5, random_state=0)

In [128]:
#vectorizer = TfidfVectorizer(min_df=1, binary=0, use_idf=1, smooth_idf=0, sublinear_tf=1)

In [131]:
from sklearn.decomposition import TruncatedSVD
pipe = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('svd', TruncatedSVD()),
    ('clf', SVC()),
])

param_grid = dict(svd__n_components=[5, 10, 50],
                   clf__kernel=['linear','rbf'],
                   clf__C=[0.001, 1, 100]
                 )

In [132]:
print("# Tuning hyper-parameters...")
print()

clf = GridSearchCV(pipe, param_grid)
clf.fit(X_train, y_train)

print("Best parameters set found on development set:")
print()
print(clf.best_params_)
print()
print("Grid scores on development set:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
          % (mean, std * 2, params))

# Tuning hyper-parameters...

Best parameters set found on development set:

{'clf__C': 100, 'clf__kernel': 'linear', 'svd__n_components': 10}

Grid scores on development set:

0.520 (+/-0.003) for {'clf__C': 0.001, 'clf__kernel': 'linear', 'svd__n_components': 5}
0.520 (+/-0.003) for {'clf__C': 0.001, 'clf__kernel': 'linear', 'svd__n_components': 10}
0.520 (+/-0.003) for {'clf__C': 0.001, 'clf__kernel': 'linear', 'svd__n_components': 50}
0.520 (+/-0.003) for {'clf__C': 0.001, 'clf__kernel': 'rbf', 'svd__n_components': 5}
0.520 (+/-0.003) for {'clf__C': 0.001, 'clf__kernel': 'rbf', 'svd__n_components': 10}
0.520 (+/-0.003) for {'clf__C': 0.001, 'clf__kernel': 'rbf', 'svd__n_components': 50}
0.923 (+/-0.027) for {'clf__C': 1, 'clf__kernel': 'linear', 'svd__n_components': 5}
0.957 (+/-0.083) for {'clf__C': 1, 'clf__kernel': 'linear', 'svd__n_components': 10}
0.984 (+/-0.009) for {'clf__C': 1, 'clf__kernel': 'linear', 'svd__n_components': 50}
0.924 (+/-0.022) for {'clf__C': 1, 'clf__kerne

In [133]:
print("Detailed classification report on test set:")
print()
y_true, y_pred = y_test, clf.predict(X_test)
print(classification_report(y_true, y_pred))
print()

print("Detailed classification report on training set")
print()
y_true, y_pred = y_train, clf.predict(X_train)
print(classification_report(y_true, y_pred))
print()

Detailed classification report on test set:

             precision    recall  f1-score   support

 accounting       1.00      1.00      1.00       160
      legal       0.97      1.00      0.99       106
        ops       1.00      0.99      0.99       291

avg / total       0.99      0.99      0.99       557


Detailed classification report on training set

             precision    recall  f1-score   support

 accounting       1.00      0.99      1.00       164
      legal       0.98      1.00      0.99       103
        ops       1.00      1.00      1.00       289

avg / total       1.00      1.00      1.00       556


